In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
#import xgboost as xgb
from sklearn.metrics import f1_score
#import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler
import cv2
from random import shuffle


%matplotlib inline
%pylab inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Populating the interactive namespace from numpy and matplotlib


In [2]:
df_train = pd.read_csv('data/train_v2.csv')
labels = df_train['tags'].apply(lambda x: x.split(' '))

df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
sample_submission = pd.read_csv('data/sample_submission_v2.csv')


In [4]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
weather = []

for i in [l.split(' ') for l in df_train['tags'].values]:
    for j in i:
        if j in weather_labels:
            weather += [j]

df_train['weather'] = weather

In [5]:
skf = StratifiedKFold(4, shuffle=True, random_state=1)
folds = []
for itr, ite in skf.split(df_train, df_train.weather):
    folds += [[itr, ite]]

In [6]:
folds

[[array([    0,     2,     4, ..., 40476, 40477, 40478]),
  array([    1,     3,    16, ..., 40469, 40470, 40475])],
 [array([    0,     1,     2, ..., 40476, 40477, 40478]),
  array([    4,     6,     8, ..., 40471, 40472, 40473])],
 [array([    0,     1,     3, ..., 40473, 40475, 40476]),
  array([    2,     5,    12, ..., 40474, 40477, 40478])],
 [array([    1,     2,     3, ..., 40475, 40477, 40478]),
  array([    0,     7,    13, ..., 40464, 40466, 40476])]]

In [7]:
import cv2
import numpy as np
import copy

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, Lambda, GlobalAveragePooling2D, Merge
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializations
from keras.engine import Layer, InputSpec
from keras import backend as K

import sys
sys.setrecursionlimit(3000)

class Scale(Layer):
    '''Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
                        This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self.trainable_weights = [self.gamma, self.beta]
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
        del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a', bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      name=conv_name_base + '2b', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = merge([x, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, subsample=strides,
                      name=conv_name_base + '2a', bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      name=conv_name_base + '2b', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
                             name=conv_name_base + '1', bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def resnet101_model(weights_path=None):
    '''Instantiate the ResNet101 architecture,
    # Arguments
        weights_path: path to pretrained weight file
    # Returns
        A Keras model instance.
    '''
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
      bn_axis = 3
      img_input = Input(shape=(224, 224, 3), name='data')
    else:
      bn_axis = 1
      img_input = Input(shape=(3, 224, 224), name='data')

    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,3):
      x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,23):
      x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
    x_fc = Flatten()(x_fc)
    #x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    model = Model(img_input, x_fc)
    
    # load weights
    if weights_path:
      model.load_weights(weights_path, by_name=True)

    return model

Using TensorFlow backend.


In [8]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 17 

    # Load Cifar10 data. Please implement your own load_data() module for your own dataset
    #X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)

    # Load our model
#model = resnet152_model(img_rows, img_cols, channel, num_classes)
#model = resnet152_model()
weights_path = 'resnet101_weights_tf.h5'


  # Test pretrained model
base_model = resnet101_model(weights_path)
#sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# add a global spatial average pooling layer
x = base_model.output
#x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
#x = Dense(512, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(17, activation='sigmoid', name='last_layer')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
from keras.callbacks import Callback, ModelCheckpoint

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='Adam',
              metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('resnet_101.h5', save_best_only=True, save_weights_only=True)

In [11]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

label_map = {l: i for i, l in enumerate(categories)}


#labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))
#label_map = {l: i for i, l in enumerate(labels)}
weather_label_map = {l: i for i, l in enumerate(weather_labels)}
  


In [12]:
def all_generator(tr, batch_size):
    while(1):
        shuffle(tr)
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        print(len(batches))
        batch_num = 0
        for i in batches:
            batch_num += 1
            if batch_num % 50 == 0:
                print(batch_num)
            x_train = []
            y_train = []

            for f, tags, weather in df_train.ix[i, :].values:
                img = cv2.imread('data/train-jpg/{}.jpg'.format(f))
                targets = np.zeros(17)
                for t in tags.split(' '):
                    targets[label_map[t]] = 1
                #targets[weather_label_map[weather]] = 1
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), random.choice([0, 90, 180, 270]), 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if random.choice([0, 1]) == 1:
                    img = cv2.flip( img, 0  )

                #1
                x_train.append(img)
                y_train.append(targets)  
            
            
            y_train = np.array(y_train, np.uint8)
            x_train = np.array(x_train, np.float16) / 255.0
            yield (x_train, y_train)

In [13]:
def val_generator(tr, batch_size, seed=0, shuf=1):
    while(1):
        if shuf==1:
            shuffle(tr)
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        
        for i in batches:
                
            x_train = []
            y_train = []

            for f, tags, weather in df_train.ix[i, :].values:
                img = cv2.imread('data/train-jpg/{}.jpg'.format(f))
                targets = np.zeros(17)
                for t in tags.split(' '):
                    targets[label_map[t]] = 1
                #targets[weather_label_map[weather]] = 1
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), (seed % 4)*90, 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if (seed % 2 == 1):
                    img = cv2.flip( img, 0  )
                #1
                x_train.append(img)
                y_train.append(targets)  

            
            y_train = np.array(y_train, np.uint8)
            x_train = np.array(x_train, np.float16) / 255.0
            yield (x_train, y_train)

In [14]:
batch_size = 64
model.fit_generator(all_generator(folds[0][0], batch_size),
                    samples_per_epoch=8000,
                    validation_data=val_generator(folds[0][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=10,
                    verbose=2,
                    callbacks=[model_checkpoint])

Epoch 1/10
476


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


50
100


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


545s - loss: 0.1637 - acc: 0.9429 - val_loss: 0.7616 - val_acc: 0.8308
Epoch 2/10
150
200
250
235s - loss: 0.1271 - acc: 0.9523 - val_loss: 1.3609 - val_acc: 0.9055
Epoch 3/10
300
350
236s - loss: 0.1232 - acc: 0.9528 - val_loss: 1.3080 - val_acc: 0.9048
Epoch 4/10
400
450
476
236s - loss: 0.1202 - acc: 0.9543 - val_loss: 1.2704 - val_acc: 0.8992
Epoch 5/10
50
100
150
236s - loss: 0.1135 - acc: 0.9565 - val_loss: 1.2189 - val_acc: 0.8511
Epoch 6/10
200
250
235s - loss: 0.1123 - acc: 0.9568 - val_loss: 0.7749 - val_acc: 0.8541
Epoch 7/10
300
350
400
281s - loss: 0.1132 - acc: 0.9570 - val_loss: 0.5978 - val_acc: 0.8286
Epoch 8/10
450
476
50
284s - loss: 0.1118 - acc: 0.9566 - val_loss: 0.4126 - val_acc: 0.8613
Epoch 9/10
100
150
284s - loss: 0.1100 - acc: 0.9576 - val_loss: 0.2122 - val_acc: 0.9261
Epoch 10/10
200
250
300
253s - loss: 0.1086 - acc: 0.9576 - val_loss: 0.1587 - val_acc: 0.9426


In [15]:
batch_size = 64
model.fit_generator(all_generator(folds[0][0], batch_size),
                    samples_per_epoch=8000,
                    validation_data=val_generator(folds[0][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=4,
                    verbose=2,
                    callbacks=[model_checkpoint])

476Epoch 1/4

50
100


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


256s - loss: 0.1095 - acc: 0.9572 - val_loss: 0.1145 - val_acc: 0.9570
Epoch 2/4
150
200
250
251s - loss: 0.1053 - acc: 0.9590 - val_loss: 0.1079 - val_acc: 0.9592
Epoch 3/4
300
350
252s - loss: 0.1065 - acc: 0.9587 - val_loss: 0.1061 - val_acc: 0.9604
Epoch 4/4
400
450
476
235s - loss: 0.1087 - acc: 0.9579 - val_loss: 0.1167 - val_acc: 0.9584


In [13]:
model.load_weights('resnet152_stage_2.h5')

In [16]:
#for layer in model.layers[:500]:
#    layer.trainable = False
for layer in model.layers[:]:
    layer.trainable = True

In [23]:
import numpy as np
from sklearn.metrics import fbeta_score
from keras import backend as K


def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [27]:
import keras
adam = keras.optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)


In [22]:
sgd = keras.optimizers.SGD(lr=0.000001, momentum=0.0, decay=0.0, nesterov=False)


In [28]:

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=adam,
              metrics=[fbeta])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model_checkpoint = ModelCheckpoint('resnet101_stage_3.h5', save_best_only=True, save_weights_only=True)




In [29]:
batch_size = 16
model.fit_generator(all_generator(folds[0][0], batch_size),
                    samples_per_epoch=8000,
                    validation_data=val_generator(folds[0][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=5,
                    verbose=2,
                    callbacks=[model_checkpoint])

Epoch 1/5
1899
50
100
150
200
250
300
350
400
450
500


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


706s - loss: 0.0874 - fbeta: 0.8830 - val_loss: 0.0934 - val_fbeta: 0.8797
Epoch 2/5
550
600
650
700
750
800
850
900
950
1000
672s - loss: 0.0862 - fbeta: 0.8861 - val_loss: 0.0917 - val_fbeta: 0.8803
Epoch 3/5
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
670s - loss: 0.0868 - fbeta: 0.8857 - val_loss: 0.0940 - val_fbeta: 0.8779
Epoch 4/5
1550
1600
1650
1700
1750
1800
1850
1899
50
100
670s - loss: 0.0853 - fbeta: 0.8864 - val_loss: 0.0922 - val_fbeta: 0.8808
Epoch 5/5
150
200
250
300
350
400
450
500
550
600
669s - loss: 0.0861 - fbeta: 0.8835 - val_loss: 0.0943 - val_fbeta: 0.8781


In [26]:
batch_size = 16
model.fit_generator(all_generator(folds[0][0], batch_size),
                    samples_per_epoch=8000,
                    validation_data=val_generator(folds[0][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=5,
                    verbose=2,
                    callbacks=[model_checkpoint])

Epoch 1/51899

50
100
150
200
250
300
350
400
450
500


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


659s - loss: 0.0874 - fbeta: 0.8860 - val_loss: 0.0943 - val_fbeta: 0.8783
Epoch 2/5
550
600
650
700
750
800
850
900
950
1000
633s - loss: 0.0874 - fbeta: 0.8850 - val_loss: 0.0920 - val_fbeta: 0.8828
Epoch 3/5
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
630s - loss: 0.0864 - fbeta: 0.8853 - val_loss: 0.0927 - val_fbeta: 0.8816
Epoch 4/5
1550
1600
1650
1700
1750
1800
1850
1899
50
100
630s - loss: 0.0892 - fbeta: 0.8822 - val_loss: 0.0939 - val_fbeta: 0.8775
Epoch 5/5
150
200
250
300
350
400
450
500
550
600
629s - loss: 0.0883 - fbeta: 0.8828 - val_loss: 0.0949 - val_fbeta: 0.8780


In [ ]:
1

1

In [ ]:
2

2

In [67]:
model.save_weights('fold1/resnet101.h5')

In [ ]:
1

1

In [ ]:
1

In [ ]:
1

In [35]:
2

2

In [42]:
folds

[[array([15980, 29233,  4392, ..., 18421, 23480, 15172]),
  array([  301, 20036, 30432, ..., 19143,  3925, 36324])],
 [array([    0,     1,     2, ..., 40476, 40477, 40478]),
  array([    4,     6,     8, ..., 40471, 40472, 40473])],
 [array([    0,     1,     3, ..., 40473, 40475, 40476]),
  array([    2,     5,    12, ..., 40474, 40477, 40478])],
 [array([    1,     2,     3, ..., 40475, 40477, 40478]),
  array([    0,     7,    13, ..., 40464, 40466, 40476])]]

In [30]:
batch_size = 256
val_pred = model.predict_generator(val_generator(folds[0][1], batch_size, shuf=0, seed=0), val_samples = len(folds[0][1]) )

In [38]:
val_pred = val_pred * 0

In [39]:
batch_size = 128
for i in range(8):
    val_pred_tmp = model.predict_generator(val_generator(folds[0][1], batch_size, shuf=0, seed=i), val_samples = len(folds[0][1]) )
    val_pred = val_pred + val_pred_tmp
    print(i)

0
1
2
3
4
5
6
7


In [44]:
val_pred = val_pred / 8

In [48]:
val_pred['id'] = folds[0][1]

In [51]:
val_pred = val_pred.sort_values('id')

In [53]:
del val_pred['id']

In [45]:
val_pred = pd.DataFrame(val_pred)

In [55]:
val_pred.to_csv("fold1/train.csv", index=None)

In [47]:
v=pd.read_csv('fold4/train.csv')

In [10]:
len(val_pred)

13494

In [14]:
len(y_val)

13494

In [56]:
y_all = []
for f, tags, weather in df_train.values:
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    y_all += [targets]
y_all = pd.DataFrame(y_all)

In [39]:
y_all.shape

(40479, 17)

In [20]:
len(folds[0][1])

10122

In [59]:
y_val = y_all.ix[folds[0][1].tolist()]

In [57]:
          
from sklearn.metrics import fbeta_score

#print(y_valid)
#print(p_valid)
print(fbeta_score(y_val, np.array(val_pred) > 0.2, beta=2, average='samples'))

0.580099944106


In [ ]:
0.922365582803
0.917041863094
0.913685914092
0.899545374775
0.89285374022
0.892722591666
0.889811292029
0.887170607068
0.863779970799

In [44]:
len(x_test)

61191

In [58]:
len(sample_submission)

61191

In [59]:
len(range(61191))

61191

In [60]:
def test_generator(tr, batch_size, seed=0):
    while(1):
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        
        for i in batches:
                
            x_test = []

            for f in sample_submission.ix[i, 'image_name'].values:
                img = cv2.imread('data/test-jpg/{}.jpg'.format(f))
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), (seed % 4)*90, 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if (seed % 2 == 1):
                    img = cv2.flip( img, 0  )
                #1
                x_test.append(img)

            
            x_test = np.array(x_test, np.float16) / 255.0
            yield x_test

In [ ]:
1

In [23]:
1

1

In [61]:
batch_size = 256
test_pred = model.predict_generator(test_generator(range(61191), batch_size, seed=0), val_samples = 61191)

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [62]:
result_pred = test_pred

In [ ]:
p_test = test_pred

In [54]:
import gc
gc.collect()

5924

In [63]:
batch_size = 256
for i in range(1,8):
    test_pred = model.predict_generator(test_generator(range(61191), batch_size, seed=i), val_samples = 61191 )
    result_pred = test_pred + result_pred
    print(i)

1
2
3
4
5
6
7


In [ ]:
2

In [56]:
1

1

In [64]:
result_pred = result_pred / 8

In [65]:
result_pred = pd.DataFrame(result_pred)

In [66]:
result_pred.to_csv("fold1/test.csv", index=None)

In [68]:
1

1

In [45]:
p_test = np.array(p_test) > 0.2

In [46]:
p_test[:3]

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True],
       [ True, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False]], dtype=bool)

In [47]:
label_map

{'agriculture': 11,
 'artisinal_mine': 1,
 'bare_ground': 15,
 'blooming': 3,
 'blow_down': 9,
 'clear': 16,
 'cloudy': 2,
 'conventional_mine': 4,
 'cultivation': 10,
 'habitation': 13,
 'haze': 12,
 'partly_cloudy': 5,
 'primary': 0,
 'road': 6,
 'selective_logging': 8,
 'slash_burn': 14,
 'water': 7}

In [48]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

In [49]:
res = []
for i in range(len(p_test)):
    res += [' '.join(categories[p_test[i]])]

In [33]:
len(res)

61191

In [50]:
sample_submission = pd.read_csv('data/sample_submission_v2.csv')

In [51]:
sample_submission['tags'] = res

In [52]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary cultivation agriculture clear
4,test_4,primary cloudy partly_cloudy


In [36]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary partly_cloudy cultivation agriculture ...
4,test_4,primary cloudy partly_cloudy


In [73]:
sample_submission[-5:]

,image_name,tags
61186,file_9995,primary partly_cloudy road water agriculture
61187,file_9996,primary road water cultivation agriculture hab...
61188,file_9997,primary clear
61189,file_9998,primary cloudy haze
61190,file_9999,primary partly_cloudy road water agriculture h...


In [53]:
sample_submission.to_csv("predictions/tmp13.csv", index=None)